## **Importing Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import resize
from scipy.misc import imsave
import matplotlib.image as mpimg
from cv2 import bitwise_and
from PIL import Image
import cv2

## **Upload data**

In [2]:
df = pd.read_csv('data/train.csv')
X_df = df['ImageId']
y_df = df['Malignant']
X = X_df.values
y = y_df.values

## Create matrix of images

In [3]:
def build_matrix(X,folder):
    # load images
    M = []
    for i in range(0, X.shape[0]):
        image = X[i]
        mask = cv2.imread(folder + '/im_segmentations/'  + str(image) + '_segmentation.jpg') 
        #upload segmentaion and resize (image*image_seg) after applying the mask
        #link = 'data/scaled' + '/'  + str(image) + '_segmentation.jpg'
        img_down = resize(mask,(250,250), mode='reflect',anti_aliasing = True) 
        #plt.imsave(image+'.jpg',img_down,cmap='gray')
        M.append(img_down)
    return np.asarray(M)

In [4]:
#Create training matrix
M = build_matrix(X,folder='data')

In [6]:
np.save('data/training_segm.npy',M)

In [4]:
#load input matrix
images = np.load('data/data_training_matrix.npy')

In [5]:
#load input matrix
images_segm = np.load('data/data_training_matrix_segmentaions.npy')

In [14]:
images_segm[0].shape

(250, 250)

In [24]:
images[0].T.shape

(3, 250, 250)

## Features

In [25]:
im = images[0].T
imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(imgray,127,255,0)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

error: OpenCV(4.0.0) c:\projects\opencv-python\opencv\modules\imgproc\src\color.hpp:259: error: (-2:Unspecified error) in function '__cdecl cv::CvtHelper<struct cv::Set<3,4,-1>,struct cv::Set<1,-1,-1>,struct cv::Set<0,2,5>,2>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 250


In [ ]:
len(countours)

In [ ]:
contours

In [ ]:
print(extract(M))

In [ ]:
def extract(image, mask, contour):
    """
    This method extracts Asymmetry, Border, and Diamter features along with
    lesion area, centroid, and perimeter. Performs affine transformation
    :param image: 3-d numpy array of an RGB image
    :param mask: binary image of the lesion mask image
    :param contour: list of contour points of the lesion
    :return: a list of all the features along with area, centroid,
    perimeter of the lesion, and transformed image
    """
    moments = cv2.moments(contour)
    #            contour_area = moments['m00']
    # (mH, mW) = mask.shape[:2]
    contour_area = cv2.countNonZero(mask)
    try:
        contour_centroid = [int(moments['m10'] / moments['m00']),
                            int(moments['m01'] / moments['m00'])]
        contour_perimeter = cv2.arcLength(contour, True)

        # Get max and min diameter
        rect = cv2.fitEllipse(contour)
        (x, y) = rect[0]
        (w, h) = rect[1]
        angle = rect[2]

        if w < h:
            if angle < 90:
                angle -= 90
            else:
                angle += 90
        rows, cols = mask.shape
        rot = cv2.getRotationMatrix2D((x, y), angle, 1)
        cos = np.abs(rot[0, 0])
        sin = np.abs(rot[0, 1])
        nW = int((rows * sin) + (cols * cos))
        nH = int((rows * cos) + (cols * sin))

        rot[0, 2] += (nW / 2) - cols / 2
        rot[1, 2] += (nH / 2) - rows / 2

        warp_mask = cv2.warpAffine(mask, rot, (nH, nW))
        warp_img = cv2.warpAffine(image, rot, (nH, nW))
        warp_img_segmented = cv2.bitwise_and(warp_img, warp_img,
                                             mask=warp_mask)

        im_mask, cnts, hierarchy = cv2.findContours(warp_mask, cv2.RETR_TREE,
                                                    cv2.CHAIN_APPROX_NONE)
        areas = [cv2.contourArea(c) for c in cnts]
        contour = cnts[np.argmax(areas)]
        xx, yy, nW, nH = cv2.boundingRect(contour)
        #        cv2.rectangle(warp_mask,(xx,yy),(xx+w,yy+h),(255,255,255),2)
        warp_mask = warp_mask[yy:yy + nH, xx:xx + nW]

        # get horizontal asymmetry
        flipContourHorizontal = cv2.flip(warp_mask, 1)
        flipContourVertical = cv2.flip(warp_mask, 0)

        diff_horizontal = cv2.compare(warp_mask, flipContourHorizontal,
                                      cv2.CV_8UC1)
        diff_vertical = cv2.compare(warp_mask, flipContourVertical,
                                    cv2.CV_8UC1)

        diff_horizontal = cv2.bitwise_not(diff_horizontal)
        diff_vertical = cv2.bitwise_not(diff_vertical)

        h_asym = cv2.countNonZero(diff_horizontal)
        v_asym = cv2.countNonZero(diff_vertical)

        return [{'area': int(contour_area), 'centroid': contour_centroid,
                 'perimeter': int(contour_perimeter),
                 'B': round(
                     (contour_perimeter ** 2) / (4 * np.pi * contour_area), 2),
                 'D1': max([nW, nH]), 'D2': min([nW, nH]),  # Normalize params
                 'A1': round(float(h_asym) / contour_area, 2),
                 'A2': round(float(v_asym) / contour_area, 2)},
                cv2.bitwise_not(diff_horizontal),
                cv2.bitwise_not(diff_vertical),
                warp_img_segmented]
    except:
return {}